In [ ]:
!apt-get install python3.11
!ln -sf /usr/bin/python3.11 /usr/bin/python

In [ ]:
!pip install lida

In [ ]:
!pip install llmx

In [ ]:
!pip install -U ydata-profiling

In [ ]:
!pip install pytorch-forecasting

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
path = "/content/drive/MyDrive/Colab Notebooks/Wolt/"

In [12]:
df_Daily_Activity = pd.read_csv(path + "daily_cp_activity_dataset.csv")
df_Daily_Activity.head(10)

,date,courier_partners_online,temperature,relative_humidity,precipitation
0,2021-05-01,49,18.27,0.57,0.00
1,2021-05-02,927,19.88,0.55,0.00
2,2021-05-03,40,16.88,0.60,0.00
3,2021-05-04,51,21.88,0.53,0.00
4,2021-05-05,50,21.11,0.54,0.00
5,2021-05-06,47,19.04,0.56,0.00
6,2021-05-07,49,25.61,0.50,0.00
7,2021-05-08,48,20.04,0.55,0.00
8,2021-05-09,42,13.49,0.67,2.88
9,2021-05-10,48,17.59,0.58,0.00


In [13]:
df_Daily_Activity.shape

(761, 5)

In [14]:
df_Daily_Activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     761 non-null    object 
 1   courier_partners_online  761 non-null    int64  
 2   temperature              731 non-null    float64
 3   relative_humidity        761 non-null    float64
 4   precipitation            756 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 29.9+ KB


# Change date format from text to date

In [15]:
df_Daily_Activity['date'] = pd.to_datetime(df_Daily_Activity['date'])
df_Daily_Activity['weekday'] = df_Daily_Activity['date'].dt.dayofweek

In [16]:
df_Daily_Activity.head()

,date,courier_partners_online,temperature,relative_humidity,precipitation,weekday
0,2021-05-01,49,18.27,0.57,0.0,5
1,2021-05-02,927,19.88,0.55,0.0,6
2,2021-05-03,40,16.88,0.60,0.0,0
3,2021-05-04,51,21.88,0.53,0.0,1
4,2021-05-05,50,21.11,0.54,0.0,2


In [17]:
# Check Null Values

df_Daily_Activity.isnull().sum()

date                        0
courier_partners_online     0
temperature                30
relative_humidity           0
precipitation               5
weekday                     0
dtype: int64

In [18]:
# Sort the DataFrame by date to ensure the correct order
df_Daily_Activity.sort_values(by='date', inplace=True)
df_Daily_Activity.reset_index(drop=True, inplace=True)

In [19]:
#Fill missing values
df_Daily_Activity['temperature'] = round((df_Daily_Activity['temperature'].ffill() + df_Daily_Activity['temperature'].bfill())/2, 2)

In [20]:
# Fill Missing Values
df_Daily_Activity['precipitation'].fillna(df_Daily_Activity['precipitation'].mode()[0], inplace=True)

In [22]:
df_Daily_Activity['year'] = df_Daily_Activity['date'].dt.year
df_Daily_Activity['month'] = df_Daily_Activity['date'].dt.month
df_Daily_Activity['day'] = df_Daily_Activity['date'].dt.day

In [23]:
# this didn't help


# df_Daily_Activity['day_bin'] = pd.to_datetime(df_Daily_Activity['date']).apply(
#     lambda x: (1 if 1 <= x.day <= 5
#                 else 2 if 6 <= x.day <= 10
#                 else 3 if 11 <= x.day <= 15
#                 else 4 if 16 <= x.day <= 20
#                 else 5 if 21 <= x.day <= 25
#                 else 6
#               ) )

In [24]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df_Daily_Activity)
profile

Output hidden; open in https://colab.research.google.com to view.

In [25]:
from scipy import stats

def normality_test(data):
    alpha = 0.05
    statistic, p_value = stats.shapiro(data)
    alpha = 0.05  # significance level
    if p_value > alpha:
        return True
    return False

In [26]:
def detect_Outlier(df, col):
    if normality_test(df[col].tolist()):
        UperBound = df[col].mean()+(3 * df[col].std())
        LowerBound = df[col].mean()-(3 * df[col].std())
        print(f"Lower Bound is {LowerBound} and Uper Bound is {UperBound}")
        print(f"{col.ljust(20)} has a Normal Distribution With {len(df[col][(df[col]<LowerBound) | (df[col]>UperBound)])} Outlier")
        print(df[col][(df[col]<LowerBound) | (df[col]>UperBound)])
        df[col][(df[col]<LowerBound) | (df[col]>UperBound)] = df[col].mode()[0]
    else:
        q1 = df[col].quantile(.25)
        q3 = df[col].quantile(.75)
        IQR = q3 - q1
        UperBound = q3 + (1.5 * IQR)
        LowerBound = q1 - (1.5 * IQR)
        print(f"Lower Bound is {LowerBound} and Uper Bound is {UperBound}")
        print(f"{col.ljust(20)} has NOT a Normal Distribution With {len(df[col][(df[col]<LowerBound) | (df[col]>UperBound)])} Outlier")
        print(df[col][(df[col]<LowerBound) | (df[col]>UperBound)])
        df[col][(df[col]<LowerBound) | (df[col]>UperBound)] = df[col].mode()[0]

In [ ]:
chk = ['courier_partners_online', 'temperature', 'relative_humidity', 'precipitation']
for col in chk:
    detect_Outlier(df_Daily_Activity, col)
    print('\n ----------------------------------------------------------------------- \n')

In [28]:
df_Daily_Activity.head(10)

,date,courier_partners_online,temperature,relative_humidity,precipitation,weekday,year,month,day
0,2021-05-01,49,18.27,0.57,0.0,5,2021,5,1
1,2021-05-02,69,19.88,0.55,0.0,6,2021,5,2
2,2021-05-03,40,16.88,0.60,0.0,0,2021,5,3
3,2021-05-04,51,21.88,0.53,0.0,1,2021,5,4
4,2021-05-05,50,21.11,0.54,0.0,2,2021,5,5
5,2021-05-06,47,19.04,0.56,0.0,3,2021,5,6
6,2021-05-07,49,25.61,0.50,0.0,4,2021,5,7
7,2021-05-08,48,20.04,0.55,0.0,5,2021,5,8
8,2021-05-09,42,13.49,0.67,0.0,6,2021,5,9
9,2021-05-10,48,17.59,0.58,0.0,0,2021,5,10


In [ ]:
features = ['temperature', 'relative_humidity', 'precipitation', 'weekday', 'year', 'month', 'day']
x = df_Daily_Activity[features]
y = df_Daily_Activity['courier_partners_online']
df = df_Daily_Activity.copy()

In [34]:
import pandas as pd
import torch
from torch import nn
import pytorch_lightning as pl
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

df = df_Daily_Activity.copy()
df = df.drop(columns=['date'])

# Feature Engineering
# df['is_holiday'] = df['date'].apply(lambda x: 1 if x in holidays else 0)

target = 'courier_partners_online'
cat_vars = ['weekday', 'year', 'month', 'day']

max_prediction_length = 31
max_encoder_length = 30
training_cutoff = df['date'].max() - pd.Timedelta(days=max_encoder_length + max_prediction_length - 1)

class DeliveryForecasting(pl.LightningModule):
    def __init__(self, datamodule, hidden_size=16, learning_rate=1e-3):
        super(DeliveryForecasting, self).__init__()
        self.save_hyperparameters()
        self.model = TemporalFusionTransformer.from_dataset(
            datamodule.train_dataloader().dataset,
            learning_rate=learning_rate,
            hidden_size=hidden_size,
            output_size=1,
            loss=QuantileLoss(),
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

training_data = TimeSeriesDataSet(
    df[df['date'] <= training_cutoff],
    time_idx='date',
    target=target,
    group_ids=cat_vars,
    min_encoder_length=1,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['temperature', 'relative_humidity', 'precipitation'], #, 'is_holiday'
    time_varying_known_reals=['temperature', 'relative_humidity', 'precipitation'], # , 'is_holiday'
)

validation_data = TimeSeriesDataSet.from_dataset(training_data, df[df['date'] > training_cutoff], predict=True, stop_randomization=True)

# Hyperparameter Tuning
tuner = optimize_hyperparameters(
    training_data,
    model=TemporalFusionTransformer,
    loss=QuantileLoss(),
    val_dataset=validation_data,
    n_trials=10,  # increased from 5
    max_epochs=20,  # increased from 10
    gradient_clip_val_range=(0.01, 1.0),
)

model = DeliveryForecasting(datamodule=training_data, hidden_size=int(tuner.best_params['hidden_size']), learning_rate=tuner.best_params['lr'])

trainer = pl.Trainer(max_epochs=30, gpus=1, gradient_clip_val=0.1)
trainer.fit(model, train_dataloader=training_data.to_dataloader(batch_size=32, num_workers=4))

# Model Evaluation
test_data = TimeSeriesDataSet.from_dataset(training_data, df_test, predict=True, stop_randomization=True)
test_dataloader = test_data.to_dataloader(batch_size=32, num_workers=4)
trainer.test(model, test_dataloaders=test_dataloader)

# Error Analysis
predictions_test = model(test_dataloader)
errors = predictions_test - y_test
errors_sorted = errors.sort_values()
print(errors_sorted[:10])  # print the 10 predictions with the highest error

# Model Interpretability
# (This is just a placeholder, as implementing this might require additional libraries or methods)
feature_importances = model.get_feature_importance()

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Callback, PyTorchLightningPruningCallback